In [5]:
pip install request

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request


In [6]:
pip install beautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [11]:
import time
import re
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen
from selenium.webdriver import ActionChains

driver = webdriver.Chrome('C:/Users/KSY/Desktop/chromedriver.exe')
driver.get("https://www.idus.com") #tab[0]
driver.execute_script('window.open("about:blank", "_blank");') #창 1개더 열어놓기 tab[1]
driver.execute_script('window.open("about:blank", "_blank");') 
tabs = driver.window_handles
time.sleep(3)

def scroll():
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(1)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

img_List=[]
writer_List=[]
with open ("ourdus_product.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite)
    time.sleep(3)
    driver.switch_to_window(tabs[0])
    scroll()
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    myWriter.writerow(["product_id","author_id","category_id","product_name",'product_price','product_rate','product_review_num','product_hit','product_purchase','product_option_num','delivery_price'])
       
    category_list =  soup.findAll('div',{'class':'ui_grid__cols--6'}) #모든 카테고리 목록
    
    cid = 0
    pid=0
    for i in category_list:
        cid+=1
        if cid==2:
            break
        category = i.findAll('div',{'class':'ui_grid__item'})
        category_id = cid
        category_info = i.find('a',{'class':'area-txt'})
        category_name = category_info.text
        category_url = "https://www.idus.com/" + str(category_info["href"])
        
        driver.switch_to_window(tabs[1]) #두번째 탭으로 이동
        driver.get(category_url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        
        class_list = soup.findAll('div',{'class':'ui_grid__item'})
        
        for j in class_list:
            pid+=1
            if pid==15: 
                break
            info_url = j.find({'a':'data-v-27ea337a'})
            info_url = "https://www.idus.com/"+ str(info_url["href"])
            img_url = j.find('div',{'class':'product-thumb-img'})
            img_url = img_url["style"]
            start=img_url.find('\"')
            end = img_url.find('\"',start+1)
            img_URL = str(img_url[start+1:end])
            img_List.append([img_URL])
            
            #img_URL로 로컬 저장소에 사진 저장
#             with urlopen(img_URL) as f:
#                 with open('./Ourdus/product_img/'+ str(pid)+ '.jpg',"wb") as h:
#                     img=f.read()
#                     h.write(img)
            
            driver.switch_to_window(tabs[2]) #세번째 탭으로 이동
            scroll()
            driver.get(info_url)
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
            
            writer_name = soup.find('span',{'class':'artist_card__label'}).get_text(strip=True)
            writer_name = writer_name.replace(" ","")
            writer_List.append([writer_name])
            
            product_name = soup.find('h2',{'class':'sticky_aside__produc-title'}).text
            
            detail_info = soup.find('div',{'class':'product-detail-info'})
            
            product_price = detail_info.find('strong',{'class':'sold-price'}).text.replace(",","")
            product_price=int(product_price)
            
            try:
                product_hit = detail_info.find('p',{'class':'star-count-text'}).get_text(strip=True)
                product_hit = product_hit.replace(",","")
                product_hit=int(product_hit)
            except:
                product_hit=0

            try:
                product_purchase = detail_info.find('span',{'id':'count_up'}).text
                product_purchase = product_purchase.replace(" ","").replace("명","").replace(",","")
                product_purchase = int(product_purchase)
            except:
                product_purchase=0
            
            try:
                point_info = detail_info.find('td',{'class':'data-row__content'})
                product_point = point_info.find({'span':'data-v-9e98616a'},{'class':'bold-txt'}).text
                product_point = product_point.replace(" ","").replace("최","").replace("대","").replace("P","").replace(",","")
                product_point=int(product_point)
            except:
                product_point=0
            
            try:
                product_rate = detail_info.find('span',{'class':'ui_rating fr'})
                product_rate = float(product_rate["data-value"])
                product_rate=int(product_rate*10)
                
            except:
                product_rate=0
            
            try:
                product_review = detail_info.find('div',{'style':'margin-left: 3px;'}).text
                product_review = product_review.replace(")","").replace("(","")
                product_review = int(product_review)
            except:
                product_review=0
            
#             delivery_info = detail_info.find('td',{'class':'data-row__content'})
#             product_delivery = delivery_info.find({'span':'data-v-9e98616a'}).get_text(strip=True)
#             product_delivery = product_delivery.replace(",","").replace("원","")
            
#             product_description = soup.findAll('p',{'class':'para'})
#  

            save = [pid,0,cid,product_name,product_price,product_rate,product_review,product_hit,product_purchase,3]
            print(save)
            myWriter.writerow(save)
            


driver.close()
    
 

<ipython-input-11-ceaa4e40db69>:38: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[0])
<ipython-input-11-ceaa4e40db69>:59: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[1]) #두번째 탭으로 이동
<ipython-input-11-ceaa4e40db69>:85: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[2]) #세번째 탭으로 이동


[1, 0, 1, '요거트짝꿍 딸기후르츠 수제청 생일선물', 16000, 48, 2374, 20445, 20122, 3]
[2, 0, 1, '딸기듬뿍 생과딸기청 수제청 생일선물', 14000, 48, 285, 3849, 1822, 3]
[3, 0, 1, '❤1kg만 할인 (2/15발송)❤ 딸기청 딸기우유 라떼', 14900, 49, 1489, 19303, 13582, 3]
[4, 0, 1, '고농축 생딸기시럽 수제청 딸기청 생일선물', 13500, 49, 1288, 12806, 9600, 3]
[5, 0, 1, '새콤달콤 레몬자몽청🍊🍋수제청 생일선물', 14000, 49, 149, 2010, 1487, 3]
[6, 0, 1, '미녀는 석류를 좋아해 석류청 수제청 생일선물', 14000, 49, 22, 1884, 826, 3]
[7, 0, 1, '새콤달콤 레몬청 수제청 생일선물', 14000, 48, 318, 3159, 2813, 3]
[8, 0, 1, '⛔가을겨울추천⛔배도라지대추청 수제청 생일선물 ', 18000, 48, 442, 3413, 4382, 3]
[9, 0, 1, 'JMTGR 패션후르츠청 수제청 생일선물', 14000, 49, 1160, 10276, 8692, 3]
[10, 0, 1, '[🎁2월특별할인]농장 직영! 딸기청-리얼딸기우유 ', 15000, 50, 1, 987, 74, 3]
[11, 0, 1, '요거트전용 애플망고청수제청 생일선물', 14000, 45, 127, 2541, 1359, 3]
[12, 0, 1, '새콤달콤 딸기키위청 수제청 생일선물', 14000, 49, 63, 1138, 611, 3]
[13, 0, 1, '시음공짜❤️고소한 에스프레소 커피 원액 500ml', 11980, 49, 988, 4188, 5949, 3]
[14, 0, 1, '🔥발렌타인🍫특가🔥대용량 더치커피 콜드브루 설날선물', 9900, 49, 2796, 7680, 10950, 3]


In [29]:
with open ("ourdus_product_image.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite) 
    for i in range(len(img_List)):
        myWriter.writerow(img_List[i])

        
with open ("ourdus_writer.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite) 
    for i in range(len(writer_List)):
        myWriter.writerow(writer_List[i])